Data Preparation 

In [ ]:
# data set downoad instructions from kaggle. We are using opensource temperature data available at Kaggle.
#We will download data using kaggle API at run time. To understand how kaggle API works, please visit kaggle link 
# https://www.kaggle.com/general/74235

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

Upload Kaggle JSON API credential file

In [ ]:
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"akshay0","key":"6af1c14002ef22c1871e98ae67aacaa5"}'}

In [ ]:
#Create directory to save Json credential file, copy JSON file and change permissions 
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
# download kaggle temperature dataset daily-temperature-of-major-cities
! kaggle datasets download -d sudalairajkumar/daily-temperature-of-major-cities

daily-temperature-of-major-cities.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
#Unzip downloaded file
! unzip daily-temperature-of-major-cities.zip

Archive:  daily-temperature-of-major-cities.zip
replace city_temperature.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


Prepare Enviornment- Install JDK 8, Pyspark version 3.2.1

In [ ]:
import os
# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed pyspark==3.2.1

openjdk version "1.8.0_352"
OpenJDK Runtime Environment (build 1.8.0_352-8u352-ga-1~20.04-b08)
OpenJDK 64-Bit Server VM (build 25.352-b08, mixed mode)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 KB 20.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853643 sha256=a75a72b8bb7c4da25622afdb2a6fceb265935a722a473b32078aec8029570c4a
  Stored in directory: /root/.cache/pip/wheels/58/94/83/915c9059e4b038e2d43a6058f307fe1c3e8536e5745f3b23b7
Successfully built pyspark


In [ ]:
! pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.2.1
      /_/
                        
Using Scala version 2.12.15, OpenJDK 64-Bit Server VM, 1.8.0_352
Branch HEAD
Compiled by user hgao on 2022-01-20T19:26:14Z
Revision 4f25b3f71238a00508a356591553f2dfa89f8290
Url https://github.com/apache/spark
Type --help for more information.


Importing Pyspark and creating spark session

In [ ]:
#set enviornment variable SPARK_VERSION
! export SPARK_VERSION=3.2.1
import pyspark
import pyspark.sql
from pyspark.sql import SparkSession
spark = SparkSession.builder.config('spark.ui.port', '4050').getOrCreate()

Importing data into spark

In [ ]:

from pyspark.sql import functions as F
data_path = '/content/city_temperature.csv'
data = spark.read.csv(data_path, header=True)

In [ ]:
#Inspect Schema
data.printSchema()

root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- State: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Day: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- AvgTemperature: string (nullable = true)



In [ ]:
data.head(5)

[Row(Region='Africa', Country='Algeria', State=None, City='Algiers', Month='1', Day='1', Year='1995', AvgTemperature='64.2'),
 Row(Region='Africa', Country='Algeria', State=None, City='Algiers', Month='1', Day='2', Year='1995', AvgTemperature='49.4'),
 Row(Region='Africa', Country='Algeria', State=None, City='Algiers', Month='1', Day='3', Year='1995', AvgTemperature='48.8'),
 Row(Region='Africa', Country='Algeria', State=None, City='Algiers', Month='1', Day='4', Year='1995', AvgTemperature='46.4'),
 Row(Region='Africa', Country='Algeria', State=None, City='Algiers', Month='1', Day='5', Year='1995', AvgTemperature='47.9')]

In [ ]:
data.show(5)

+------+-------+-----+-------+-----+---+----+--------------+
|Region|Country|State|   City|Month|Day|Year|AvgTemperature|
+------+-------+-----+-------+-----+---+----+--------------+
|Africa|Algeria| null|Algiers|    1|  1|1995|          64.2|
|Africa|Algeria| null|Algiers|    1|  2|1995|          49.4|
|Africa|Algeria| null|Algiers|    1|  3|1995|          48.8|
|Africa|Algeria| null|Algiers|    1|  4|1995|          46.4|
|Africa|Algeria| null|Algiers|    1|  5|1995|          47.9|
+------+-------+-----+-------+-----+---+----+--------------+
only showing top 5 rows



Lets explore and clean data using Python API functions

In [ ]:
from pyspark.sql.types import IntegerType,FloatType
data_df = data.withColumn("Avgtempcent", data["AvgTemperature"].cast(FloatType()))
data_clean=data_df.where("Avgtempcent> -99")

In [ ]:
data_clean.show(5)

+------+-------+-----+-------+-----+---+----+--------------+-----------+
|Region|Country|State|   City|Month|Day|Year|AvgTemperature|Avgtempcent|
+------+-------+-----+-------+-----+---+----+--------------+-----------+
|Africa|Algeria| null|Algiers|    1|  1|1995|          64.2|       64.2|
|Africa|Algeria| null|Algiers|    1|  2|1995|          49.4|       49.4|
|Africa|Algeria| null|Algiers|    1|  3|1995|          48.8|       48.8|
|Africa|Algeria| null|Algiers|    1|  4|1995|          46.4|       46.4|
|Africa|Algeria| null|Algiers|    1|  5|1995|          47.9|       47.9|
+------+-------+-----+-------+-----+---+----+--------------+-----------+
only showing top 5 rows



Data Transformations

In [ ]:
# But Temperature given in Farenheight, Lets convert to Celcius scale

In [ ]:
data_df1 = data_clean.withColumn("Avgtempcentconv", 5/9*(data_df["Avgtempcent"]-9))
data_df1.show(5)

+------+-------+-----+-------+-----+---+----+--------------+-----------+------------------+
|Region|Country|State|   City|Month|Day|Year|AvgTemperature|Avgtempcent|   Avgtempcentconv|
+------+-------+-----+-------+-----+---+----+--------------+-----------+------------------+
|Africa|Algeria| null|Algiers|    1|  1|1995|          64.2|       64.2|30.666664971245662|
|Africa|Algeria| null|Algiers|    1|  2|1995|          49.4|       49.4| 22.44444529215495|
|Africa|Algeria| null|Algiers|    1|  3|1995|          48.8|       48.8| 22.11111068725586|
|Africa|Algeria| null|Algiers|    1|  4|1995|          46.4|       46.4| 20.77777862548828|
|Africa|Algeria| null|Algiers|    1|  5|1995|          47.9|       47.9|21.611111958821617|
+------+-------+-----+-------+-----+---+----+--------------+-----------+------------------+
only showing top 5 rows



Data Exploration

In [ ]:
#Mean temp of all countries of Asia region for each year
data_df1.select('Region','Country','City','Year','Avgtempcentconv').where("Region=='Asia'")\
.groupBy("Country","Year").agg(F.mean('Avgtempcentconv')).orderBy("Country","Year").show(5)

+----------+----+--------------------+
|   Country|Year|avg(Avgtempcentconv)|
+----------+----+--------------------+
|Bangladesh|1995|   39.08342950301002|
|Bangladesh|1996|  38.758802764665724|
|Bangladesh|1997|   37.86378861750306|
|Bangladesh|1998|  39.048365898381654|
|Bangladesh|1999|  37.826599336232384|
+----------+----+--------------------+
only showing top 5 rows



In [ ]:
#Mean temp of all countries of Asia region for each year
data_df1.select('Region','Country','City','Year','Avgtempcentconv').where("Region=='Asia'")\
.groupBy("Country","Year").agg(F.mean('Avgtempcentconv')).orderBy("Country","Year").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [Country#416 ASC NULLS FIRST, Year#421 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(Country#416 ASC NULLS FIRST, Year#421 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#590]
      +- HashAggregate(keys=[Country#416, Year#421], functions=[avg(Avgtempcentconv#535)])
         +- Exchange hashpartitioning(Country#416, Year#421, 200), ENSURE_REQUIREMENTS, [id=#587]
            +- HashAggregate(keys=[Country#416, Year#421], functions=[partial_avg(Avgtempcentconv#535)])
               +- Project [Country#416, Year#421, (cast((cast(AvgTemperature#422 as float) - 9.0) as double) * 0.5555555555555556) AS Avgtempcentconv#535]
                  +- Filter (((isnotnull(AvgTemperature#422) AND isnotnull(Region#415)) AND (cast(AvgTemperature#422 as float) > -99.0)) AND (Region#415 = Asia))
                     +- FileScan csv [Region#415,Country#416,Year#421,AvgTemperature#422] Batched: false, DataFilters: [isnotnull(AvgTe

Using SQL in Spark Jobs

In [ ]:
#Lets create& register a view on our dataframe
data_df1.createOrReplaceTempView("Temperature")

In [ ]:
#Lets Run sql Query to fetch average temperature of countries for each year of Asia Region
df=spark.sql("select Country,Year, AVG(Avgtempcentconv) from Temperature where region='Asia' group by 1,2 order by 1,2")
df.show(5)

+----------+----+--------------------+
|   Country|Year|avg(Avgtempcentconv)|
+----------+----+--------------------+
|Bangladesh|1995|   39.08342950301002|
|Bangladesh|1996|  38.758802764665724|
|Bangladesh|1997|   37.86378861750306|
|Bangladesh|1998|  39.048365898381654|
|Bangladesh|1999|  37.826599336232384|
+----------+----+--------------------+
only showing top 5 rows



OK, does we have same explain plan as before??

In [ ]:
df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [Country#17 ASC NULLS FIRST, Year#22 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(Country#17 ASC NULLS FIRST, Year#22 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#673]
      +- HashAggregate(keys=[Country#17, Year#22], functions=[avg(Avgtempcentconv#136)])
         +- Exchange hashpartitioning(Country#17, Year#22, 200), ENSURE_REQUIREMENTS, [id=#670]
            +- HashAggregate(keys=[Country#17, Year#22], functions=[partial_avg(Avgtempcentconv#136)])
               +- Project [Country#17, Year#22, (cast((cast(AvgTemperature#23 as float) - 9.0) as double) * 0.5555555555555556) AS Avgtempcentconv#136]
                  +- Filter (((isnotnull(AvgTemperature#23) AND isnotnull(region#16)) AND (cast(AvgTemperature#23 as float) > -99.0)) AND (region#16 = Asia))
                     +- FileScan csv [Region#16,Country#17,Year#22,AvgTemperature#23] Batched: false, DataFilters: [isnotnull(AvgTemperature#23), isnotn

In [ ]:
#Average temperature of all countries
df=spark.sql("select  AVG(Avgtempcentconv) from Temperature ").show()

+--------------------+
|avg(Avgtempcentconv)|
+--------------------+
|  28.541046728168215|
+--------------------+



Does Spark allows Complex SQL queries e.g. CTEs , Subqueries ?


In [ ]:
#Get all Asian countries, year and corresponding Average temp  having their Avg temp higher than avg global temperature till now 
spark.sql(""" \
WITH T_avg AS ( \
    select avg(Avgtempcentconv) as av from Temperature \
), \
CY_AVG AS (select Country,Year, AVG(Avgtempcentconv) as atc from Temperature where region='Asia' group by 1,2) \
select * from CY_AVG WHERE atc >= (SELECT av from T_avg) order by atc desc""" ).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [atc#304 DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(atc#304 DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [id=#318]
      +- Filter (isnotnull(atc#304) AND (atc#304 >= Subquery subquery#302, [id=#310]))
         :  +- Subquery subquery#302, [id=#310]
         :     +- AdaptiveSparkPlan isFinalPlan=false
         :        +- HashAggregate(keys=[], functions=[avg(Avgtempcentconv#136)])
         :           +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#308]
         :              +- HashAggregate(keys=[], functions=[partial_avg(Avgtempcentconv#136)])
         :                 +- Project [(cast((cast(AvgTemperature#23 as float) - 9.0) as double) * 0.5555555555555556) AS Avgtempcentconv#136]
         :                    +- Filter (isnotnull(AvgTemperature#23) AND (cast(AvgTemperature#23 as float) > -99.0))
         :                       +- FileScan csv [AvgTemperature#23] Batched: false, DataFilters:

In [ ]:
#Lets check the data of above query

In [ ]:
spark.sql(""" \
WITH T_avg AS ( \
    select avg(Avgtempcentconv) as av from Temperature \
), \
CY_AVG AS (select Country,Year, AVG(Avgtempcentconv) as atc from Temperature where region='Asia' group by 1,2) \
select * from CY_AVG WHERE atc >= (SELECT av from T_avg) order by atc desc""" ).show(5)

+---------+----+------------------+
|  Country|Year|               atc|
+---------+----+------------------+
| Thailand|1997| 42.77184178368322|
|Indonesia|2002| 42.48148186496957|
| Thailand|2006| 42.29984775816104|
| Thailand|2005| 42.25281578075578|
|Indonesia|2004|42.102481097208006|
+---------+----+------------------+
only showing top 5 rows



In [ ]:
#Average temperature of all countries
df=spark.sql("DESCRIBE Temperature ").show()

+---------------+---------+-------+
|       col_name|data_type|comment|
+---------------+---------+-------+
|         Region|   string|   null|
|        Country|   string|   null|
|          State|   string|   null|
|           City|   string|   null|
|          Month|   string|   null|
|            Day|   string|   null|
|           Year|   string|   null|
| AvgTemperature|   string|   null|
|    Avgtempcent|    float|   null|
|Avgtempcentconv|   double|   null|
+---------------+---------+-------+



Lets explore Spark UI .To Enable UI in Google Collab, you need to set up ngrok auth which will provide a publlic URL for you SPARK UI . To set up your token, go to https://dashboard.ngrok.com/get-started/your-authtoken

In [ ]:
#findspark will allow to locate spark installation in your enviornment
!pip install -q findspark

In [ ]:
#import and initialize findspark
import findspark
findspark.init()

In [ ]:
#get your spark installation directory
findspark.find()

'/usr/local/lib/python3.8/dist-packages/pyspark'

In [ ]:
#Download and install Ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
#Unzip installer
!unzip ngrok-stable-linux-amd64.zip
#Setup your authtoken
!./ngrok authtoken <Your Authtoken>
#generate public URL
get_ipython().system_raw('./ngrok http 4050 &')


--2023-02-04 06:01:33--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.237.133.81, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  40.7MB/s    in 0.3s    

2023-02-04 06:01:34 (40.7 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
#Locate public URL
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://ac01-34-67-227-98.ngrok.io","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://ac01-34-67-227-98.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}
